In [31]:
# on a besoin de path_to_code pour pouvoir importer paths.py, le serpent se mort la queue :D
path_to_code = 'C:/Nicolas/M2 MVA/ALTEGRAD/Kaggle/text_and_graph/code'
# path_to_code = "/Users/estelleaflalo/Desktop/M2_Data_Science/Second_Period/Text_and_Graph/Project/text_and_graph/code/"

import sys

sys.path.append(path_to_code)

from paths import path

path_to_code, path_to_data, path_to_results = path("nicolas")

import numpy as np
from init import split, init_dic, csv_to_sub
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from numpy.linalg import norm
from loss_function import score
%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
training = pd.read_csv(path_to_data + 'training_set.csv', sep=',', header=0)

training_info = pd.read_csv(path_to_data + 'training_info.csv', sep=',', header=0)

test = pd.read_csv(path_to_data + 'test_set.csv', sep=',', header=0)

test_info = pd.read_csv(path_to_data + 'test_info.csv', sep=',', header=0)

print "Building dictionnaries"

_, all_senders, _, address_books, _ = init_dic(training, training_info)

Building dictionnaries
0


10


20


30


40


...

80


90


100


110


120


In [4]:
def get_all_recs_per_sender(train_info_S):
    """
    For a fixed sender, return the list of all the recipients of this sender in the training set.
    :param train_info_S: A structure like train_info, but for one sender only, since we work sender by sender
    :return: a list of all the recipients for the current sender
    """
    all_recs = []
    for recs in train_info_S['recipients'].values:
        all_recs.extend(recs.split(' '))

    all_recs = list(set(all_recs))
    return all_recs


X_train, Y_train, X_test = csv_to_sub(training, training_info, test, test_info)

In [7]:
use_idf = True
print 'Parameter use_idf is set to {}'.format(use_idf)


K=20


print 'parameter K is set to {}'.format(K)
max_df = 0.95
min_df = 1
print 'To build the vocabulary, the tfidfVectorizer will use max_df={} and min_df={}'.format(max_df, min_df)
sublinear_tf  = True # default is False in sklearn
if sublinear_tf:
    print 'The tf is replaced by (1 + log(tf))'

Parameter use_idf is set to True
parameter K is set to 20
To build the vocabulary, the tfidfVectorizer will use max_df=0.95 and min_df=1
The tf is replaced by (1 + log(tf))


In [16]:
index=1
sender = all_senders[index]
X_train_S = X_train[sender]
X_dev_S = X_test[sender]
Y_train_S = Y_train[sender]

##############Create TF IDF vector from mails sent by sender S


# vectorize mails sent by a unique sender
vectorizer_sender = TfidfVectorizer(max_df=0.95, stop_words='english', use_idf=use_idf, sublinear_tf=sublinear_tf)

# train
training_info_S = training_info.loc[training_info['mid'].isin(X_train_S)]
training_info_S = training_info_S.set_index(np.arange(len(training_info_S)))
training_info_S_mat = training_info_S.as_matrix()
content_train = training_info_S_mat[:, 2]

vec_train = vectorizer_sender.fit_transform(content_train)
bow_train = vec_train.toarray()
# test
test_info_S = test_info.loc[test_info['mid'].isin(X_dev_S)]
test_info_S = test_info_S.set_index(np.arange(len(test_info_S)))
test_info_S_mat = test_info_S.as_matrix()
content_test = test_info_S_mat[:, 2]

vec_test = vectorizer_sender.transform(content_test)
bow_test = vec_test.toarray()

In [32]:
address_books[sender]
n_recs = len(address_books[sender])
print n_recs
Adj = np.concatenate((np.ones((n_recs+1,1)), np.zeros((n_recs+1, n_recs))), axis=1)
Adj[0,0] = 0
G = nx.from_numpy_matrix(Adj)
pos=nx.circular_layout(G)
#nx.draw_networkx_labels(G,pos,labels=names,font_size_lab=2, alpha=0.5, font_family='sans-serif')
nx.draw_circular(G, node_color='r')
plt.show()

147


In [20]:
all_recs = get_all_recs_per_sender(training_info_S)
all_recs_name = []
for recs in all_recs:
    all_recs_name.append(recs.split('@')[0])
print all_recs_name
print len(all_recs)

['susan.mara', 'linda.noske', 'wade.cline', 'patrick.keene', 'jose.bestard', 'ray.alvarez', 'carolyn.cooney', 'tom.chapman', 'barbara.hueter', 'brendan.devlin', 'sandra', 'steve.burns', 'dan.staines', 'l..petrochko', 'm..landwehr', 'richard.shapiro', 'susan.scott', 'vinio.floris', 'robert.frank', 'laurie.knight', 'doug.wood', 'pat.shortridge', 'robert.hemstock', 'susan.landwehr', 'alan.comnes', 'l..nicolay', 'ron.mcnamara', 'guillermo.canovas', 'merle.glen', 'dave.mangskau', 'harry.kingerski', 'tom.fitzgibbon', 'teun.biert', 'bryan.gottfredson', 'maria.arefieva', 'lisa.yoho', 'nailia.dindarova', 'nancy.hetrick', 'margaret.huson', 'howard.fromer', 'frank.rishe', 'john.shelk', 'lara.leibman', 'kerry.stroup', 'jan.haizmann', 'paul.hennemeyer', 'larry.decker', 'elizabeth.linnell', 'miyung.buster', 'paul.kaufman', 'alfredo.huertas', 'jean.dressler', 'lysa.tracy', 'daniel.allegretti', 'rubena.buerger', 'alisha.nathoo', 'allen.joe', 'gisele.braz', 'takashi.kimura', 'linda.robertson', 'steve.m